In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
7,"COVID-19: Equitable vaccine access for all, in...",https://reliefweb.int/report/world/covid-19-eq...,"Sat, 23 Jan 2021 03:22:00 GMT",2021-01-23 03:22:00+00:00
8,Executive Order on Economic Relief Related to ...,https://www.whitehouse.gov/briefing-room/presi...,"Fri, 22 Jan 2021 22:17:21 GMT",2021-01-22 22:17:21+00:00
6,Houston Health Department opening new COVID-19...,https://www.houstontx.gov/health/NewsReleases/...,"Fri, 22 Jan 2021 22:07:54 GMT",2021-01-22 22:07:54+00:00
1,TDH Updates COVID-19 Vaccination Plan - tn.gov...,https://www.tn.gov/health/news/2021/1/22/tdh-u...,"Fri, 22 Jan 2021 20:07:45 GMT",2021-01-22 20:07:45+00:00
0,"Coronavirus (COVID-19) Update: January 22, 202...",https://www.fda.gov/news-events/press-announce...,"Fri, 22 Jan 2021 20:00:00 GMT",2021-01-22 20:00:00+00:00
3,"In response to third COVID surge, Louisiana la...",https://ldh.la.gov/index.cfm/newsroom/detail/5997,"Fri, 22 Jan 2021 18:18:05 GMT",2021-01-22 18:18:05+00:00
4,Episode #22 - Children & COVID-19 World Healt...,https://www.who.int/emergencies/diseases/novel...,"Fri, 22 Jan 2021 16:00:19 GMT",2021-01-22 16:00:19+00:00
5,COVID test pooling hampered by high positivity...,https://www.modernhealthcare.com/supply-chain/...,"Fri, 22 Jan 2021 15:43:19 GMT",2021-01-22 15:43:19+00:00
2,KFF COVID-19 Vaccine Monitor: January 2021 | K...,https://www.kff.org/coronavirus-covid-19/repor...,"Fri, 22 Jan 2021 13:56:20 GMT",2021-01-22 13:56:20+00:00
9,COVID-19 Drug Scorecard Is Mixed : Shots - Hea...,https://www.npr.org/sections/health-shots/2021...,"Fri, 22 Jan 2021 10:06:00 GMT",2021-01-22 10:06:00+00:00
